In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds


In [4]:
import cv2

In [6]:
ls


 Volume in drive C is Windows-SSD
 Volume Serial Number is 2CD7-88B3

 Directory of C:\Users\KIIT\projects\cvpr

29-10-2024  14:22    <DIR>          .
16-10-2024  23:35    <DIR>          ..
29-10-2024  14:17    <DIR>          .ipynb_checkpoints
29-10-2024  14:17    <DIR>          class
29-10-2024  14:17           114,506 CVPR_cbam_project.ipynb
29-10-2024  14:21            53,768 hasan_1.jpg
29-10-2024  14:21            19,871 hasan_2.jpg
29-10-2024  14:21           234,210 manodeep.jpg
               4 File(s)        422,355 bytes
               4 Dir(s)  62,163,431,424 bytes free


In [15]:
pic1 = cv2.imread("manodeep.jpg")
#pic1 = cv2.cvtColor(pic1, cv2.COLOR_BGR2GRAY)
pic2 = cv2.imread("hasan_1.jpg")


In [16]:
pic1 = cv2.resize(pic1,(224,224))
pic2 = cv2.resize(pic2,(224,224))
pic1.shape,pic2.shape

((224, 224, 3), (224, 224, 3))

In [18]:
classes = {1:"manodeep" , 2:"hasan"}

In [19]:
'''import tensorflow as tf
from tensorflow.keras import layers, models

# Channel Attention Module (CAM)
def channel_attention(input_feature, ratio=8):
    channel = input_feature.shape[-1]  # Get the number of channels

    # Shared Dense layers
    shared_layer_one = layers.Dense(channel // ratio, activation='relu', kernel_initializer='he_normal')
    shared_layer_two = layers.Dense(channel, kernel_initializer='he_normal')

    # Global Average Pooling
    avg_pool = layers.GlobalAveragePooling2D()(input_feature)
    avg_pool = layers.Reshape((1, 1, channel))(avg_pool)
    avg_pool = shared_layer_one(avg_pool)
    avg_pool = shared_layer_two(avg_pool)

    # Global Max Pooling
    max_pool = layers.GlobalMaxPooling2D()(input_feature)
    max_pool = layers.Reshape((1, 1, channel))(max_pool)
    max_pool = shared_layer_one(max_pool)
    max_pool = shared_layer_two(max_pool)

    # Add & Activate (sigmoid)
    cbam_feature = layers.Add()([avg_pool, max_pool])
    cbam_feature = layers.Activation('sigmoid')(cbam_feature)

    return layers.Multiply()([input_feature, cbam_feature])  # Scale the input with attention weights

# Spatial Attention Module (SAM)
def spatial_attention(input_feature):
    # Average Pooling & Max Pooling along the channel axis
    avg_pool = layers.Lambda(lambda x: tf.reduce_mean(x, axis=-1, keepdims=True))(input_feature)
    max_pool = layers.Lambda(lambda x: tf.reduce_max(x, axis=-1, keepdims=True))(input_feature)

    # Concatenate along the last axis (channel axis)
    concat = layers.Concatenate(axis=-1)([avg_pool, max_pool])

    # Apply a 7x7 Conv2D layer followed by sigmoid activation
    cbam_feature = layers.Conv2D(filters=1, kernel_size=7, strides=1, padding='same', activation='sigmoid', kernel_initializer='he_normal')(concat)

    return layers.Multiply()([input_feature, cbam_feature])  # Scale input by spatial attention

# CBAM Block (Combines both CAM and SAM)
def cbam_block(input_feature, ratio=8):
    # Apply Channel Attention Module (CAM)
    x = channel_attention(input_feature, ratio)

    # Apply Spatial Attention Module (SAM)
    x = spatial_attention(x)

    return x

# CNN model with CBAM attention
def create_cnn_with_attention(input_shape=(224, 224, 3), num_classes=2):
    inputs = layers.Input(shape=input_shape)

    # Initial CNN Layers
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)

    # CBAM Block after the first convolutional block
    x = cbam_block(x)

    # Continue with CNN Layers
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Another CBAM Block
    x = cbam_block(x)

    # Flatten and add Dense Layers
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)

    # Output layer for classification
    class_output = layers.Dense(num_classes, activation='softmax')(x)

    # Create the full model
    model = models.Model(inputs=inputs, outputs=class_output)

    return model'''

# Create the CNN model with CBAM attention
model = create_cnn_with_attention()

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()

# Example training dataset preparation (replace with actual dataset)
# Assuming `train_dataset` is a tf.data.Dataset object
# model.fit(train_dataset, epochs=10)





Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 224, 224, 128)        3584      ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 112, 112, 128)        0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 global_average_pooling2d (  (None, 128)                  0         ['max_pooling2d[0][0]']

                                                                                                  
 lambda_2 (Lambda)           (None, 56, 56, 1)            0         ['multiply_2[0][0]']          
                                                                                                  
 lambda_3 (Lambda)           (None, 56, 56, 1)            0         ['multiply_2[0][0]']          
                                                                                                  
 concatenate_1 (Concatenate  (None, 56, 56, 2)            0         ['lambda_2[0][0]',            
 )                                                                   'lambda_3[0][0]']            
                                                                                                  
 conv2d_3 (Conv2D)           (None, 56, 56, 1)            99        ['concatenate_1[0][0]']       
                                                                                                  
 multiply_

In [36]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras import backend as K
import numpy as np

# Define the base CNN model for feature extraction
'''def create_base_cnn(input_shape=(224, 224, 3)):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu')  # Final embedding layer
    ])
    return model
'''
import tensorflow as tf
from tensorflow.keras import layers, models

# Channel Attention Module (CAM)
def channel_attention(input_feature, ratio=8):
    channel = input_feature.shape[-1]  # Get the number of channels

    # Shared Dense layers
    shared_layer_one = layers.Dense(channel // ratio, activation='relu', kernel_initializer='he_normal')
    shared_layer_two = layers.Dense(channel, kernel_initializer='he_normal')

    # Global Average Pooling
    avg_pool = layers.GlobalAveragePooling2D()(input_feature)
    avg_pool = layers.Reshape((1, 1, channel))(avg_pool)
    avg_pool = shared_layer_one(avg_pool)
    avg_pool = shared_layer_two(avg_pool)

    # Global Max Pooling
    max_pool = layers.GlobalMaxPooling2D()(input_feature)
    max_pool = layers.Reshape((1, 1, channel))(max_pool)
    max_pool = shared_layer_one(max_pool)
    max_pool = shared_layer_two(max_pool)

    # Add & Activate (sigmoid)
    cbam_feature = layers.Add()([avg_pool, max_pool])
    cbam_feature = layers.Activation('sigmoid')(cbam_feature)

    return layers.Multiply()([input_feature, cbam_feature])  # Scale the input with attention weights

# Spatial Attention Module (SAM)
def spatial_attention(input_feature):
    # Average Pooling & Max Pooling along the channel axis
    avg_pool = layers.Lambda(lambda x: tf.reduce_mean(x, axis=-1, keepdims=True))(input_feature)
    max_pool = layers.Lambda(lambda x: tf.reduce_max(x, axis=-1, keepdims=True))(input_feature)

    # Concatenate along the last axis (channel axis)
    concat = layers.Concatenate(axis=-1)([avg_pool, max_pool])

    # Apply a 7x7 Conv2D layer followed by sigmoid activation
    cbam_feature = layers.Conv2D(filters=1, kernel_size=7, strides=1, padding='same', activation='sigmoid', kernel_initializer='he_normal')(concat)

    return layers.Multiply()([input_feature, cbam_feature])  # Scale input by spatial attention

# CBAM Block (Combines both CAM and SAM)
def cbam_block(input_feature, ratio=8):
    # Apply Channel Attention Module (CAM)
    x = channel_attention(input_feature, ratio)

    # Apply Spatial Attention Module (SAM)
    x = spatial_attention(x)

    return x

# CNN model with CBAM attention
def create_cnn_with_attention(input_shape=(224, 224, 3)):
    inputs = layers.Input(shape=input_shape)

    # Initial CNN Layers
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)

    # CBAM Block after the first convolutional block
    x = cbam_block(x)

    # Continue with CNN Layers
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Another CBAM Block
    x = cbam_block(x)

    # Flatten and add Dense Layers
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)

    '''# Output layer for classification
    class_output = layers.Dense(num_classes, activation='softmax')(x)'''

    # Create the full model
    model = models.Model(inputs=inputs, outputs=x)

    return model

# Define the Euclidean distance function
def euclidean_distance(vectors):
    (featsA, featsB) = vectors
    sum_squared = K.sum(K.square(featsA - featsB), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_squared, K.epsilon()))

# Define the Siamese network
def create_siamese_network(input_shape=(224, 224, 3)):
    # Two inputs for the two branches
    inputA = layers.Input(shape=input_shape)
    inputB = layers.Input(shape=input_shape)

    # Shared CNN base model
    base_model = create_cnn_with_attention(input_shape=( 224, 224, 3))

    # Generate the feature vectors for both inputs
    featsA = base_model(inputA)
    featsB = base_model(inputB)

    # Compute the Euclidean distance between the two feature vectors
    distance = layers.Lambda(euclidean_distance)([featsA, featsB])

    # Final layer to classify similarity: 0 (dissimilar) or 1 (similar)
    output = layers.Dense(1, activation='sigmoid')(distance)

    # Create the Siamese network model
    model = models.Model(inputs=[inputA, inputB], outputs=output)

    return model

# Create the Siamese network
siamese_model = create_siamese_network()

# Compile the model
siamese_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
siamese_model.summary()

# Example training dataset (replace with your actual dataset)
# For Siamese networks, inputs are typically pairs of images with a label indicating similarity.
# Assuming `train_pairs` is the input data in the format [(img1, img2), label].
#labels = 1
#train_pairs = [(pic2, pic1), label]# where label is 0 or 1.

pic1 = cv2.imread("manodeep.jpg")
#pic1 = cv2.cvtColor(pic1, cv2.COLOR_BGR2GRAY)
pic2 = cv2.imread("hasan_1.jpg")


pic1 = cv2.resize(pic1, (224, 224)) / 255.0  # Normalize and resize to match input shape
pic2 = cv2.resize(pic2, (224, 224)) / 255.0
'''pic1 = np.expand_dims(pic1, axis=0)  # Shape becomes (1, 224, 224, 3)
pic2 = np.expand_dims(pic2, axis=0)  # Shape becomes (1, 224, 224, 3)'''
# Create pairs of images as separate arrays
img1_data = np.array([pic1,pic2])  # Wrap in array and add more pairs as needed
img2_data = np.array([pic1,pic2])

# Create labels as an array (binary values: 0 for dissimilar, 1 for similar)
labels = np.array([1 , 1])  # For example, label as 1 (similar) or 0 (dissimilar)

# Now train the model with arrays of image pairs and labels

siamese_model.fit([img1_data, img2_data], labels, epochs=10)
#siamese_model.fit([pic1, pic1], labels, epochs=10)


Model: "model_23"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_38 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 input_39 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 model_22 (Functional)       (None, 128)                  2577315   ['input_38[0][0]',            
                                                          0          'input_39[0][0]']            
                                                                                                  
 lambda_62 (Lambda)          (None, 1)                    0         ['model_22[0][0]',     

In [37]:
'''import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, backend as K
from tensorflow.keras.datasets import cifar10
from sklearn.model_selection import train_test_split

# Load and preprocess MNIST dataset
def load_mnist_data():
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    # Normalize the images to [0, 1] and expand dimensions for channels
    x_train = np.expand_dims(x_train / 255.0, axis=-1)
    x_test = np.expand_dims(x_test / 255.0, axis=-1)

    return x_train, y_train, x_test, y_test
import tensorflow as tf
from tensorflow.keras import layers, models

# Channel Attention Module (CAM)
def channel_attention(input_feature, ratio=8):
    channel = input_feature.shape[-1]  # Get the number of channels

    # Shared Dense layers
    shared_layer_one = layers.Dense(channel // ratio, activation='relu', kernel_initializer='he_normal')
    shared_layer_two = layers.Dense(channel, kernel_initializer='he_normal')

    # Global Average Pooling
    avg_pool = layers.GlobalAveragePooling2D()(input_feature)
    avg_pool = layers.Reshape((1, 1, channel))(avg_pool)
    avg_pool = shared_layer_one(avg_pool)
    avg_pool = shared_layer_two(avg_pool)

    # Global Max Pooling
    max_pool = layers.GlobalMaxPooling2D()(input_feature)
    max_pool = layers.Reshape((1, 1, channel))(max_pool)
    max_pool = shared_layer_one(max_pool)
    max_pool = shared_layer_two(max_pool)

    # Add & Activate (sigmoid)
    cbam_feature = layers.Add()([avg_pool, max_pool])
    cbam_feature = layers.Activation('sigmoid')(cbam_feature)

    return layers.Multiply()([input_feature, cbam_feature])  # Scale the input with attention weights

# Spatial Attention Module (SAM)
def spatial_attention(input_feature):
    # Average Pooling & Max Pooling along the channel axis
    avg_pool = layers.Lambda(lambda x: tf.reduce_mean(x, axis=-1, keepdims=True))(input_feature)
    max_pool = layers.Lambda(lambda x: tf.reduce_max(x, axis=-1, keepdims=True))(input_feature)

    # Concatenate along the last axis (channel axis)
    concat = layers.Concatenate(axis=-1)([avg_pool, max_pool])

    # Apply a 7x7 Conv2D layer followed by sigmoid activation
    cbam_feature = layers.Conv2D(filters=1, kernel_size=7, strides=1, padding='same', activation='sigmoid', kernel_initializer='he_normal')(concat)

    return layers.Multiply()([input_feature, cbam_feature])  # Scale input by spatial attention

# CBAM Block (Combines both CAM and SAM)
def cbam_block(input_feature, ratio=8):
    # Apply Channel Attention Module (CAM)
    x = channel_attention(input_feature, ratio)

    # Apply Spatial Attention Module (SAM)
    x = spatial_attention(x)

    return x

# CNN model with CBAM attention
def create_cnn_with_attention(input_shape=(32, 32, 3), num_classes=10):
    inputs = layers.Input(shape=input_shape)

    # Initial CNN Layers
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)

    # CBAM Block after the first convolutional block
    x = cbam_block(x)

    # Continue with CNN Layers
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Another CBAM Block
    x = cbam_block(x)

    # Flatten and add Dense Layers
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)

    # Output layer for classification
    class_output = layers.Dense(num_classes, activation='softmax')(x)

    # Create the full model
    model = models.Model(inputs=inputs, outputs=class_output)

    return model



# Example training dataset preparation (replace with actual dataset)
# Assuming `train_dataset` is a tf.data.Dataset object
# model.fit(train_dataset, epochs=10)

# Create pairs of images for training Siamese network
def create_pairs(x, y):
    pairs = []
    labels = []
    num_classes = len(np.unique(y))
    digit_indices = [np.where(y == i)[0] for i in range(num_classes)]

    for idx in range(len(x)):
        current_image = x[idx]
        current_label = y[idx]

        # Positive pair (same class)
        positive_idx = np.random.choice(digit_indices[current_label])
        positive_image = x[positive_idx]

        pairs.append([current_image, positive_image])
        labels.append(1)  # Label for similar images is 1

        # Negative pair (different class)
        negative_label = np.random.choice([i for i in range(num_classes) if i != current_label])
        negative_idx = np.random.choice(digit_indices[negative_label])
        negative_image = x[negative_idx]

        pairs.append([current_image, negative_image])
        labels.append(0)  # Label for dissimilar images is 0

    return np.array(pairs), np.array(labels)

# Define the base CNN model for feature extraction
def create_base_cnn(input_shape=(28, 28, 1)):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu')  # Final embedding layer
    ])
    return model

# Define the Euclidean distance function
def euclidean_distance(vectors):
    (featsA, featsB) = vectors
    sum_squared = K.sum(K.square(featsA - featsB), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_squared, K.epsilon()))

# Define the Siamese network
def create_siamese_network(input_shape=(28, 28, 1) , num_classes = 10):
    # Two inputs for the two branches
    inputA = layers.Input(shape=input_shape)
    inputB = layers.Input(shape=input_shape)

    # Shared CNN base model
    #base_model = create_base_cnn(input_shape)
    base_model = create_cnn_with_attention(input_shape , num_classes)
    # Generate the feature vectors for both inputs
    featsA = base_model(inputA)
    featsB = base_model(inputB)

    # Compute the Euclidean distance between the two feature vectors
    distance = layers.Lambda(euclidean_distance)([featsA, featsB])

    # Final layer to classify similarity: 0 (dissimilar) or 1 (similar)
    output = layers.Dense(1, activation='sigmoid')(distance)

    # Create the Siamese network model
    model = models.Model(inputs=[inputA, inputB], outputs=output)

    return model

# Load the MNIST dataset
x_train, y_train, x_test, y_test = load_mnist_data()

# Create pairs of images and their corresponding labels for training and testing
train_pairs, train_labels = create_pairs(x_train, y_train)
test_pairs, test_labels = create_pairs(x_test, y_test)

# Split pairs into two sets of inputs for the Siamese network
train_pair_1, train_pair_2 = train_pairs[:, 0], train_pairs[:, 1]
test_pair_1, test_pair_2 = test_pairs[:, 0], test_pairs[:, 1]

# Create the Siamese network model
siamese_model = create_siamese_network()

# Compile the model
siamese_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
siamese_model.summary()

# Train the Siamese network on the MNIST pairs
siamese_model.fit([train_pair_1, train_pair_2], train_labels,
                  validation_data=([test_pair_1, test_pair_2], test_labels),
                  epochs=10, batch_size=64)

# Evaluate the model
loss, accuracy = siamese_model.evaluate([test_pair_1, test_pair_2], test_labels)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")
'''

'import numpy as np\nimport tensorflow as tf\nfrom tensorflow.keras import layers, models, backend as K\nfrom tensorflow.keras.datasets import cifar10\nfrom sklearn.model_selection import train_test_split\n\n# Load and preprocess MNIST dataset\ndef load_mnist_data():\n    (x_train, y_train), (x_test, y_test) = cifar10.load_data()\n\n    # Normalize the images to [0, 1] and expand dimensions for channels\n    x_train = np.expand_dims(x_train / 255.0, axis=-1)\n    x_test = np.expand_dims(x_test / 255.0, axis=-1)\n\n    return x_train, y_train, x_test, y_test\nimport tensorflow as tf\nfrom tensorflow.keras import layers, models\n\n# Channel Attention Module (CAM)\ndef channel_attention(input_feature, ratio=8):\n    channel = input_feature.shape[-1]  # Get the number of channels\n\n    # Shared Dense layers\n    shared_layer_one = layers.Dense(channel // ratio, activation=\'relu\', kernel_initializer=\'he_normal\')\n    shared_layer_two = layers.Dense(channel, kernel_initializer=\'he_norm

In [38]:
'''
# Train the Siamese network on the MNIST pairs


# Evaluate the model
loss, accuracy = siamese_model.evaluate([test_pair_1, test_pair_2], test_labels)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")'''

'\n# Train the Siamese network on the MNIST pairs\n\n\n# Evaluate the model\nloss, accuracy = siamese_model.evaluate([test_pair_1, test_pair_2], test_labels)\nprint(f"Test Loss: {loss}, Test Accuracy: {accuracy}")'

In [22]:
siamese_model.summary()


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 model_2 (Functional)        (None, 2)                    2577340   ['input_5[0][0]',             
                                                          8          'input_6[0][0]']             
                                                                                                  
 lambda_12 (Lambda)          (None, 1)                    0         ['model_2[0][0]',       

In [47]:
test_img1 = cv2.imread('hasan_1.jpg')
test_img2 = cv2.imread('hasan_2.jpg')
test_img1 = cv2.resize(test_img1, (224, 224)) / 255.0  # Scale pixel values to [0, 1]
test_img2 = cv2.resize(test_img2, (224, 224)) / 255.0

# Expand dimensions to simulate a batch of 1 sample per image
test_img1 = np.expand_dims(test_img1, axis=0)  # Shape: (1, 224, 224, 3)
test_img2 = np.expand_dims(test_img2, axis=0)  # Shape: (1, 224, 224, 3)

# Make a similarity prediction
similarity_score = siamese_model.predict([test_img1, test_img2])

# Interpret the result
if similarity_score[0][0] > 0.5:
    print("The images are similar.")
else:
    print("The images are dissimilar.")


1/1 [==============================] - 0s 56ms/step
The images are similar.


In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, backend as K
import numpy as np
from tensorflow.keras.datasets import mnist

# Load and prepare the MNIST data
(input_train, label_train), (input_test, label_test) = mnist.load_data()
input_train, input_test = input_train / 255.0, input_test / 255.0  # Normalize to [0, 1]

# Resize to fit model input (28, 28, 1) for grayscale images
input_train = np.expand_dims(input_train, -1)
input_test = np.expand_dims(input_test, -1)

# Function to create pairs of images and labels (1 for similar, 0 for dissimilar)
def create_pairs(images, labels, num_pairs=10000):
    pairs = []
    pair_labels = []
    num_classes = 10
    digit_indices = [np.where(labels == i)[0] for i in range(num_classes)]

    for _ in range(num_pairs):
        # Select a random digit
        digit = np.random.randint(0, num_classes)
        idx1, idx2 = np.random.choice(digit_indices[digit], 2, replace=False)
        # Add a similar pair
        pairs += [[images[idx1], images[idx2]]]
        pair_labels += [1]

        # Add a dissimilar pair
        other_digit = (digit + np.random.randint(1, num_classes)) % num_classes
        idx1, idx2 = np.random.choice(digit_indices[digit], 1), np.random.choice(digit_indices[other_digit], 1)
        pairs += [[images[idx1][0], images[idx2][0]]]
        pair_labels += [0]

    return np.array(pairs), np.array(pair_labels)

# Generate training and testing pairs
train_pairs, train_labels = create_pairs(input_train, label_train, num_pairs=60000)
test_pairs, test_labels = create_pairs(input_test, label_test, num_pairs=10000)

# Channel Attention Module (CAM)
def channel_attention(input_feature, ratio=8):
    channel = input_feature.shape[-1]
    shared_layer_one = layers.Dense(channel // ratio, activation='relu', kernel_initializer='he_normal')
    shared_layer_two = layers.Dense(channel, kernel_initializer='he_normal')

    avg_pool = layers.GlobalAveragePooling2D()(input_feature)
    avg_pool = layers.Reshape((1, 1, channel))(avg_pool)
    avg_pool = shared_layer_one(avg_pool)
    avg_pool = shared_layer_two(avg_pool)

    max_pool = layers.GlobalMaxPooling2D()(input_feature)
    max_pool = layers.Reshape((1, 1, channel))(max_pool)
    max_pool = shared_layer_one(max_pool)
    max_pool = shared_layer_two(max_pool)

    cbam_feature = layers.Add()([avg_pool, max_pool])
    cbam_feature = layers.Activation('sigmoid')(cbam_feature)
    return layers.Multiply()([input_feature, cbam_feature])

# Spatial Attention Module (SAM)
def spatial_attention(input_feature):
    avg_pool = layers.Lambda(lambda x: tf.reduce_mean(x, axis=-1, keepdims=True))(input_feature)
    max_pool = layers.Lambda(lambda x: tf.reduce_max(x, axis=-1, keepdims=True))(input_feature)
    concat = layers.Concatenate(axis=-1)([avg_pool, max_pool])
    cbam_feature = layers.Conv2D(filters=1, kernel_size=7, strides=1, padding='same', activation='sigmoid', kernel_initializer='he_normal')(concat)
    return layers.Multiply()([input_feature, cbam_feature])

# CBAM Block (Combines CAM and SAM)
def cbam_block(input_feature, ratio=8):
    x = channel_attention(input_feature, ratio)
    x = spatial_attention(x)
    return x

# Define the CNN model with CBAM attention for MNIST
def create_cnn_with_attention(input_shape=(28, 28, 1)):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = cbam_block(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = cbam_block(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    return models.Model(inputs, x)

# Define Euclidean distance function
def euclidean_distance(vectors):
    featsA, featsB = vectors
    sum_squared = K.sum(K.square(featsA - featsB), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_squared, K.epsilon()))

# Define Siamese model with CBAM CNN base
def create_siamese_network(input_shape=(28, 28, 1)):
    inputA = layers.Input(shape=input_shape)
    inputB = layers.Input(shape=input_shape)

    base_cnn = create_cnn_with_attention(input_shape)
    featsA = base_cnn(inputA)
    featsB = base_cnn(inputB)

    distance = layers.Lambda(euclidean_distance)([featsA, featsB])
    output = layers.Dense(1, activation="sigmoid")(distance)

    return models.Model(inputs=[inputA, inputB], outputs=output)

# Compile and train the model
siamese_model = create_siamese_network()
siamese_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
siamese_model.fit(
    [train_pairs[:, 0], train_pairs[:, 1]], train_labels,
    validation_data=([test_pairs[:, 0], test_pairs[:, 1]], test_labels),
    batch_size=32,
    epochs=2
)

# Test prediction on a test pair
test_pred = siamese_model.predict([test_pairs[0:1, 0], test_pairs[0:1, 1]])
print("Similarity score:", test_pred[0][0])






Epoch 1/2


1248/3750 [========>.....................] - ETA: 3:47 - loss: 0.6932 - accuracy: 0.4975

KeyboardInterrupt: 